In [1]:
import pandas as pd

# Шлях до файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\landcover_result.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Виведення назв стовпців
print("Назви стовпців:")
print(df.columns)

Назви стовпців:
Index(['X', 'Y', 'elevation', 'acq_time', 'frp', 'acq_date', 'confidence',
       'latitude', 'scan', 'instrument', 'time_sprea', 'type', 'slope',
       'version', 'brightness', 'group_id', 'bright_t31', 'daynight', 'id',
       'satellite', 'track', 'Map', 'longitude'],
      dtype='object')


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# Шлях до нового файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_final.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Вибір потрібних ознак і цільової змінної
features = ['elevation', 'acq_time', 'frp', 'confidence', 'slope', 'brightness', 'bright_t31', 'Map']
target = 'distance_to_nearest'

# Виділення підмножини даних
data = df[features + [target]].copy()

# Розділення на тренувальний та тестовий набір
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Вибірки ознак та цільової змінної
X_train = train_data[features]
y_train = train_data[target].fillna(0)  # Заповнюємо NaN значення в цільовій змінній нулями

X_test = test_data[features]
y_test = test_data[target].fillna(0)  # Заповнюємо NaN значення в цільовій змінній нулями

# Обробка null значень
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Побудова моделі випадкового лісу
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_imputed, y_train)

# Передбачення
predictions = rf_model.predict(X_test_imputed)

# Оцінка моделі
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Виведення результатів
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(results.head())

Mean Squared Error: 18.967437676192965
         Actual  Predicted
2323   0.000000   1.674908
18111  0.000000   6.233357
10259  2.861962   3.311373
15942  0.000000   4.981366
13889  0.000000   2.042039


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Шлях до нового файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_final.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Вибір потрібних ознак і цільової змінної
features = ['elevation', 'acq_time', 'frp', 'confidence', 'slope', 'brightness', 'bright_t31', 'Map']
target = 'distance_to_nearest'

# Вилучення рядків, де 'distance_to_nearest' містить NaN
df = df.dropna(subset=[target])

# Розділення на тренувальний та тестовий набір
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Вибірки ознак та цільової змінної
X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

# Обробка null значень
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Побудова моделі випадкового лісу
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_imputed, y_train)

# Передбачення
predictions = rf_model.predict(X_test_imputed)

# Оцінка моделі
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

# Статистичні параметри цільової змінної
print(f'Statistical Parameters for {target}:')
print(f'Mean: {y_test.mean()}')
print(f'Median: {y_test.median()}')
print(f'Standard Deviation: {y_test.std()}')
print(f'Variance: {y_test.var()}')
print(f'Minimum Value: {y_test.min()}')
print(f'Maximum Value: {y_test.max()}')

# Статистичні параметри передбачень
print(f'\nStatistical Parameters for Predictions:')
print(f'Mean: {predictions.mean()}')
print(f'Median: {pd.Series(predictions).median()}')
print(f'Standard Deviation: {pd.Series(predictions).std()}')
print(f'Variance: {pd.Series(predictions).var()}')
print(f'Minimum Value: {pd.Series(predictions).min()}')
print(f'Maximum Value: {pd.Series(predictions).max()}')

Mean Absolute Error: 3.628971337774225
Statistical Parameters for distance_to_nearest:
Mean: 5.966562978508045
Median: 2.59486144037297
Standard Deviation: 6.34241750895222
Variance: 40.22625985786368
Minimum Value: 0.020532537759681
Maximum Value: 19.997420317480373

Statistical Parameters for Predictions:
Mean: 6.122934682440525
Median: 6.492246767518496
Standard Deviation: 4.183676546070605
Variance: 17.503149442141265
Minimum Value: 0.3503962975259017
Maximum Value: 16.412719172633853


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Шлях до нового файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_final.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Вибір потрібних ознак і цільової змінної
features = ['elevation', 'acq_time', 'frp', 'confidence', 'slope', 'brightness', 'bright_t31', 'Map']
target = 'distance_to_nearest'

# Вилучення рядків, де 'distance_to_nearest' містить NaN
df = df.dropna(subset=[target])

# Розділення на тренувальний та тестовий набір
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Вибірки ознак та цільової змінної
X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

# Обробка null значень
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Параметри для Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Побудова моделі Random Forest
rf_model = RandomForestRegressor(random_state=42)

# Grid Search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_imputed, y_train)

# Отримання кращих параметрів
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Передбачення з кращими параметрами
best_rf_model = grid_search.best_estimator_
predictions = best_rf_model.predict(X_test_imputed)

# Оцінка моделі
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error with Best Parameters: {mae}')

# Виведення результатів
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(results.head())

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Absolute Error with Best Parameters: 3.6335435366924287
          Actual  Predicted
7717   13.918246  10.591405
15402   1.271352  10.703757
1805    4.119499   7.501541
5553    1.283224   7.450547
18816   0.898213   3.723566


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Шлях до нового файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_final.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Вибір потрібних ознак і цільової змінної
features = ['elevation', 'acq_time', 'frp', 'confidence', 'slope', 'brightness', 'bright_t31', 'Map']
target = 'distance_to_nearest'

# Вилучення рядків, де 'distance_to_nearest' містить NaN
df = df.dropna(subset=[target])

# Розділення на тренувальний та тестовий набір
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Вибірки ознак та цільової змінної
X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

# Обробка null значень
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Побудова та навчання моделі Random Forest з найкращими параметрами
rf_model = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

rf_model.fit(X_train_imputed, y_train)

# Передбачення
predictions = rf_model.predict(X_test_imputed)

# Оцінка моделі
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error with Best Parameters: {mae}')

# Виведення результатів
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(results.head())

Mean Absolute Error with Best Parameters: 3.6213004939723223
          Actual  Predicted
7717   13.918246  10.421731
15402   1.271352  10.490225
1805    4.119499   7.701637
5553    1.283224   7.381803
18816   0.898213   3.973178


In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

# Шлях до нового файлу CSV
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_final.csv"

# Завантаження датасету
df = pd.read_csv(file_path)

# Вибір потрібних ознак і цільової змінної
features = ['elevation', 'acq_time', 'frp', 'confidence', 'slope', 'brightness', 'bright_t31', 'Map']
target = 'distance_to_nearest'

# Вилучення рядків, де 'distance_to_nearest' містить NaN
df = df.dropna(subset=[target])

# Вибірки ознак та цільової змінної
X = df[features]
y = df[target]

# Обробка null значень
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Побудова та навчання моделі Random Forest з найкращими параметрами
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

rf_model.fit(X_imputed, y)

# Передбачення
predictions = rf_model.predict(X_imputed)

# Додавання стовпця з передбаченнями до оригінального датасету
df['Predicted_distance'] = predictions





KeyboardInterrupt: 

In [18]:
# Збереження результатів у новий CSV файл
output_file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_predictions.csv"
df.to_csv(output_file_path, index=False)

In [20]:
file_path = r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\Пожежі 2-14 днів\full data\selected_fires_with_distance_predictions.csv"
# Завантаження датасету
df = pd.read_csv(file_path)

In [21]:
# Створення нового стовпця із значеннями абсолютної різниці між фактичною і передбаченою відстанню
df['Absolute_Error'] = abs(df['distance_to_nearest'] - df['Predicted_distance'])

# Виведення інформації про пожежі з великою похибкою
large_errors = df[df['Absolute_Error'] > 7]
print(large_errors)

           X          Y  elevation  acq_time   frp    acq_date  confidence  \
2944  3.8125  51.161902        8.0      1148  30.3  2020-07-31          45   

      latitude  scan instrument  ...  daynight     id  satellite  track   Map  \
2944   51.1619   2.8      MODIS  ...         D  36170       Aqua    1.6  50.0   

      longitude  nearest distance_to_nearest  Predicted_distance  \
2944     3.8125  36229.0            19.75635           12.618335   

     Absolute_Error  
2944       7.138015  

[1 rows x 27 columns]
